# La Quinte Juste.


Ce Notebook propose une étude mathématique de La Quinte Juste, épisode de la série Kaamelott d'Alexandre Astier
Cet extrait est visible par exemple via le lien suivant :
https://www.symphozik.info/video-la-quinte-juste-269.html
L'objet de ce notebook est plus précisément de visualiser dans le domaine de Fourier les liens qui existent entre les différentes notes. Nous verrons qu'une note est dite à la quinte d'une autre si sa fréquence fondamentale vaut les deux tiers de celle de la note de référence et qu'une note est à la quarte si sa fréquence fondamentale est au 3/4 de  celle de la note de référence. Ces observations peuvent se faire directement sur les coefficients de Fourier discrets des extraits sonores.
Pour une vidéo explicative sur le lien plus précis entre mathématiques et notes de musique, pour comprendre que les quintes sur nos instruments modernes ne sont pas justes, justement pour éviter que certaines tierces sonnent trop faux et pourquoi au bas moyen âge où on ne favorisait que les quartes et les quintes, les tierces étaient particulièrement 
dissonnantes : cet épisode de la science étonnante : https://www.youtube.com/watch?v=cTYvCpLRwao

In [ ]:
import holoviews as hv
hv.extension('bokeh')
import numpy as np
import param
import holoviews as hv,panel as pn,param
from holoviews.streams import Pipe
import time
import pandas as pd
import panel as pn
from panel.pane import LaTeX
from scipy.io.wavfile import read
from scipy import fftpack
from IPython.display import Audio
import requests
from io import BytesIO
from PIL import Image
import shutil
from urllib.request import urlopen
import io
from scipy.io.wavfile import read
import scipy.io as sio
from holoviews import streams

Nous extrayons d'abord de la bande sonore, les parties qui nous intéressent.

### Extraction des sons

In [ ]:
test="parties_a_exploiter.wav"
quintejuste=read(test)
print(np.shape(quintejuste[1][:,0]))
audio_quinte=quintejuste[1][:,0]/2**14
Fe=44100
Audio(audio_quinte,rate=Fe)

In [ ]:
basse=audio_quinte[142300:165400]
Audio(basse,rate=Fe)

In [ ]:
unisson=audio_quinte[404800:450200]
Audio(unisson,rate=Fe)

In [ ]:
quarte=audio_quinte[817900:940200]
#unisson : quarte[:50000]
#quarte : quarte[70000:]
Audio(quarte,rate=Fe)

In [ ]:
quinte=audio_quinte[1087500:1200000]
#unisson : quinte[:60000]
#quinte : quinte[80000:]
Audio(quinte,rate=Fe)

In [ ]:
tierce=audio_quinte[3425700:3500700]
Audio(tierce,rate=Fe)

In [ ]:
triton=audio_quinte[6869600:]
Audio(triton,rate=Fe)

### Affichage de chaque signal brut

Nous affichons ensuite dans le domaine temporel les différents signaux. Vous pouvez zoomer pour mieux les voir. Ils sont localement stationnaires, c'est-à-dire que leur contenu fréquentiel varie peu dans le temps. 

In [ ]:
hv.Curve(unisson).opts(width=800,title='Unisson')

In [ ]:
hv.Curve(quarte).opts(width=800,title='Quarte')

In [ ]:
hv.Curve(quinte).opts(width=800,title='Quinte')

In [ ]:
hv.Curve(tierce).opts(width=800,title='Tierce')

In [ ]:
hv.Curve(triton).opts(width=800,title='Triton')

### Analyse des spectrogrammes

Attention, les vecteurs associés à la quarte et quinte contiennent une partie d'unisson

In [ ]:
nmax=4096#294*10
fft_unisson=abs(fftpack.fft(unisson[:nmax]))
fft_quarte=abs(fftpack.fft(quarte[70000:][:nmax]))
fft_quinte=abs(fftpack.fft(quinte[80000:][:nmax]))
fft_tierce=abs(fftpack.fft(tierce[:nmax]))
fft_triton=abs(fftpack.fft(triton[:nmax]))

Pour afficher le spectrogramme associé à chaque son on peut utiliser la fonction suivante $AffichageSpectre$ qui renvoie directement une sortie graphique. Le paramètre $fmax$ permet d'afficher le spectrogramme seulement jusqu'à la fréquence $fmax$.

In [ ]:
def AffichageSpectre(t,fmax=44100):
    temp=t[np.linspace(0,44100,len(t))<fmax]
    n=np.linspace(0,fmax,len(temp))
    
    return hv.Spikes(np.c_[n,temp],'Fréquence(hz)',' ').opts(width=800,height=200)

def aff_interactif(x,y):
    return hv.VLine(x)*hv.Text(x+0.05, y, '%.3f'% x, halign='left', valign='bottom')

On va maintenant afficher les modules des transformées de Fourier discrètes. Les fréquences ont ici été affichées en Herz... mais il faut garder en tête qu'on représente bien des modules de coefficients de TF discrète. Nous verrons que le lien entre l'indice $k$ et la fréquence en Hz est donné par la formule suivante :
\begin{equation}
Freq_k=\frac{Freq_{echan}*k}{N}
\end{equation}
où $Freq_{echan}$ est la fréquence d'échantillonnage, $k$ l'indice du coefficient de Fourier et $N$ le nombre de points sur lequel est effectué la trasnformée. Ainsi si on échantillonne un signal à 44100 Hz et qu'on effectue une Tf discrète sur 1000 points, le coefficient de Fourier d'indice 20 correspond à une fréquence de 882Hz. 

In [ ]:
plot_unisson=AffichageSpectre(fft_unisson,1000)
pointer = streams.PointerXY(x=0, y=0, source=plot_unisson)
plot_unisson*hv.DynamicMap(aff_interactif, streams=[pointer])

les graphiques suivants permettent de comparer visuellement les différentes fréquences présentes dans les différents sons. 

In [ ]:
nmax=-1
fft_unisson=abs(fftpack.fft(unisson[:nmax]))
fft_quarte=abs(fftpack.fft(quarte[70000:][:nmax]))
fft_quinte=abs(fftpack.fft(quinte[80000:][:nmax]))
fft_tierce=abs(fftpack.fft(tierce[:nmax]))
fft_triton=abs(fftpack.fft(triton[:nmax]))
fmax=1000
plot_unisson=AffichageSpectre(fft_unisson,fmax)
plot_quarte=AffichageSpectre(fft_quarte,fmax)
plot_quinte=AffichageSpectre(fft_quinte,fmax)
plot_tierce=AffichageSpectre(fft_tierce,fmax)
plot_triton=AffichageSpectre(fft_triton,fmax)
# pointer_unisson=streams.PointerXY(x=0, y=0, source=plot_unisson)
# pointer_quarte=streams.PointerXY(x=0, y=0, source=plot_quarte)
# pointer_quinte=streams.PointerXY(x=0, y=0, source=plot_quinte)
# pointer_tierce=streams.PointerXY(x=0, y=0, source=plot_tierce)
# pointer_triton=streams.PointerXY(x=0, y=0, source=plot_triton)
pn.Column(plot_unisson.opts(title='Unisson'),\
          plot_quarte.opts(title='Quarte'),\
          plot_quinte.opts(title='Quinte'),\
          plot_tierce.opts(title='Tierce'),\
          plot_triton.opts(title='Triton'))

Dans les graphiques suivants, les fréquences les plus présentes ont été indiquées en couleur. Vous pouvez voir que la note fondamentale de la basse est à environ 150Hz et que les multiples de cette fréquences sont aussi présentes. on dit que ces multiples sont les harmoniques de la note fondamentale. La fréquence de la note fondamentale de la quarte vaut les 3/4 de celle de la note précédente... ce qui fait que certaines harmoniques sont communes, celles à 450 et à 900Hz. C'est cette conjnction des harmoniques qui rend le son agréable à l'oreille. Si vous passez la souris sur le graphique associé à la quarte, vous pourrez effcectuer une comparaison en temps réel avec le graphique de l'unisson.   

In [ ]:
nmax=-1#16950
fft_unisson=abs(fftpack.fft(unisson[:nmax]))
fft_quarte=abs(fftpack.fft(quarte[70000:][:nmax]))
plot_unisson=AffichageSpectre(fft_unisson,fmax)
plot_quarte=AffichageSpectre(fft_quarte,fmax)
plot_quarte2=AffichageSpectre(fft_quarte,fmax)
pointer_unisson=streams.PointerXY(x=0, y=0, source=plot_unisson)
pointer_quarte=streams.PointerXY(x=0, y=0, source=plot_quarte)
pn.Column(plot_unisson.opts(title='Unisson')*hv.DynamicMap(aff_interactif, streams=[pointer_quarte]),\
          plot_quarte.opts(title='Quarte')*hv.DynamicMap(aff_interactif, streams=[pointer_quarte]),\
          plot_unisson.opts(title='Unisson')*hv.VLine(150).opts(color='red')*hv.VLine(300).opts(color='magenta')*hv.VLine(450).opts(color='magenta')*hv.VLine(600).opts(color='magenta')*hv.VLine(750).opts(color='magenta')*hv.VLine(900).opts(color='magenta'),\
          plot_quarte2.opts(title='Quarte')*hv.VLine(150).opts(color='red')*hv.VLine(300).opts(color='magenta')*hv.VLine(448).opts(color='magenta')*hv.VLine(600).opts(color='magenta')*hv.VLine(750).opts(color='magenta')*hv.VLine(898).opts(color='magenta')*hv.VLine(112.5).opts(color='blue')*hv.VLine(225).opts(color='cyan')*hv.VLine(337.5).opts(color='cyan')*hv.VLine(452).opts(color='cyan')*hv.VLine(562.5).opts(color='cyan')*hv.VLine(675).opts(color='cyan')*hv.VLine(787.5).opts(color='cyan')*hv.VLine(902).opts(color='cyan'))

Le même phénomène se produit avec la quinte et cette fois ci un rapport 2/3 entre les fréquences. Cette fois ci la première harmonique commune est à 300Hz et il y a aussi 600Hz et 900Hz. Vous pourrez noter qu'il y a en fait un petit décaclage qui s'accentue... ce peut être dû au fait que ce n'est pas une quinte juste... mais une quinte dans une game tempérée, qui est la gamme actuelle où le rapport de fréquences entre une note et sa quinte n'est pas 3/2 mais ... $2^{7/12}\approx 1.498$.


In [ ]:
nmax=-1#16950
fft_unisson=abs(fftpack.fft(unisson[:nmax]))
fft_quinte=abs(fftpack.fft(quinte[80000:][:nmax]))
plot_unisson=AffichageSpectre(fft_unisson,fmax)
plot_quinte=AffichageSpectre(fft_quinte,fmax)
plot_quinte2=AffichageSpectre(fft_quinte,fmax)
pointer_unisson=streams.PointerXY(x=0, y=0, source=plot_unisson)
pointer_quinte=streams.PointerXY(x=0, y=0, source=plot_quinte)
pn.Column(plot_unisson.opts(title='Unisson')*hv.DynamicMap(aff_interactif, streams=[pointer_quinte]),\
          plot_quinte.opts(title='Quinte')*hv.DynamicMap(aff_interactif, streams=[pointer_quinte]),\
          plot_quinte2.opts(title='Quinte')*hv.VLine(150).opts(color='red')*hv.VLine(298).opts(color='magenta')*hv.VLine(450).opts(color='magenta')*hv.VLine(598).opts(color='magenta')*hv.VLine(750).opts(color='magenta')*hv.VLine(900).opts(color='magenta')*hv.VLine(100).opts(color='blue')*hv.VLine(200).opts(color='cyan')*hv.VLine(302).opts(color='cyan')*hv.VLine(400).opts(color='cyan')*hv.VLine(500).opts(color='cyan')*hv.VLine(602).opts(color='cyan')*hv.VLine(700).opts(color='cyan')*hv.VLine(800).opts(color='cyan'))